In [1]:
# start up again with our simple example
from databaker.framework import *
tabs = loadxlstabs("example1.xls")
tab = tabs[0]  # there is only one table
savepreviewhtml(tabs[0])


Loading example1.xls which has size 7168 bytes
Table names: ['beatles', 'stones']


Date,2014.0,,
,,,
,Cars,Planes,Trains
John,2.0,2.0,1.0
Paul,4xxxx,3.0,2.0
Ringo,4.0,1.0,3.0
George,2.0,5.0,5.0


In [2]:
# A table is also "bag of cells", which just so happens to be a set of all the 
# cells in the table.  

# A "bag of cells" is like a Python set (and looks like one when you print it), 
# but it has various selection functions that help you navigate around the table

# There will be a full list of these functions in the next tutorial, but for now
# here are some examples:
# https://sensiblecodeio.github.io/quickcode-ons-docs/lang/function_reference.html

bb = tab.is_bold()
print("The bold cells in this table are", bb)
print("The", len(bb), "cells below these bold cells are", bb.shift(DOWN))
cc = tab.filter("Cars")
cc.assert_one()
print("The cell that says 'Cars' is", cc)
print("Everything in the column below the 'Cars' cell is", cc.fill(DOWN))
print("If you wanted to include the 'Cars' heading, then use expand", cc.expand(DOWN))



The bold cells in this table are {<D3 'Trains'>, <C3 'Planes'>}
The 2 cells below these bold cells are {<C4 2.0>, <D4 1.0>}
The cell that says 'Cars' is {<B3 'Cars'>}
Everything in the column below the 'Cars' cell is {<B5 '4xxxx'>, <B6 4.0>, <B7 2.0>, <B4 2.0>}
If you wanted to include the 'Cars' heading, then use expand {<B5 '4xxxx'>, <B6 4.0>, <B3 'Cars'>, <B7 2.0>, <B4 2.0>}


In [3]:
# Note that the bags are printed as unordered sets, which can make them hard to read.
# This is a feature of the underlying messytables library.  However, if you can't stand 
# it, it is possible to print them in tabular order by converting the underlying 
# unordered set into an ordered list and sorting it
hcc = tab.filter("Cars").expand(DOWN)
sorted(hcc.unordered_cells, key=lambda Cell:(Cell.y, Cell.x))

[<B3 'Cars'>, <B4 2.0>, <B5 '4xxxx'>, <B6 4.0>, <B7 2.0>]

In [4]:
# Although, sometimes it's easier just to preview it in tabular form using this helper function
savepreviewhtml(hcc)

In [5]:
# As you work through this tutorial, do please feel free to temporarily insert 
# new Jupyter-Cells in order to give yourself a place to experiment with any 
# of the functions that are available.  (Remember, the value of the last line in a 
# Jupyter-Cell is always printed out -- in addition to any earlier print-statements.)
print("All the cells that have an 'o' in them:", tab.regex(".*?o"))


All the cells that have an 'o' in them: {<A4 'John'>, <A6 'Ringo'>, <A7 'George'>}


In [6]:
# Now we begin the actual worked example, which shows you how we find the cells 
# that contain the observations, match them to the cells that are the headings 
# in the tables, and then extract their values

# We can select bags of of cells from this tab and plot them using savepreviewhtml(), like so:
savepreviewhtml(tab.excel_ref('B1'))

In [7]:
# It's important to select the observations as one of the sets
obs = tab.excel_ref('B4').expand(DOWN).expand(RIGHT).is_not_blank().is_not_whitespace()
savepreviewhtml(obs)

In [8]:
# You can also print the bag of unordered cells in one of these sets
# There are 12 here
obs


{<C4 2.0>, <B5 '4xxxx'>, <C7 5.0>, <C6 1.0>, <B4 2.0>, <D7 5.0>, <D4 1.0>, <C5 3.0>, <D6 3.0>, <B6 4.0>, <B7 2.0>, <D5 2.0>}

In [9]:
# It can be useful to look at two cell bags at once with different colours
r1 = tab.excel_ref('B3').expand(RIGHT)
r2 = tab.excel_ref('A3').fill(DOWN)
savepreviewhtml([r1, r2])  # <-- here we pass in a list containing two cell bags
print("r1 is", r1)
print("r2 is", r2)

r1 is {<D3 'Trains'>, <B3 'Cars'>, <C3 'Planes'>}
r2 is {<A4 'John'>, <A6 'Ringo'>, <A5 'Paul'>, <A7 'George'>}


In [10]:
# An HDim is an object made from a bag of cells corresponding to a set of headings, a name, 
# and some instructions on how to match the correct heading cell from 
# an observation cell.  

# The HDim constructor's arguments are: (cellbag, name, DIRECTLY|CLOSEST, *direction*)
h1 = HDim(r1, "Vehicles", DIRECTLY, ABOVE)

# Let's take a single cell and matches its heading using the cellvalobs function
cc = tab.excel_ref('C4')
print("Cell", cc, "matches", h1.cellvalobs(cc), "in dimension", h1.label)


Cell {<C4 2.0>} matches (<C3 'Planes'>, 'Planes') in dimension Vehicles


In [11]:
# You can start to see through to the final result of all this work when you 
# print out the lookup values for every observation in the table at once.  
for ob in obs:
    print("Obs", ob, "maps to", h1.cellvalobs(ob))

Obs {<B4 2.0>} maps to (<B3 'Cars'>, 'Cars')
Obs {<C4 2.0>} maps to (<C3 'Planes'>, 'Planes')
Obs {<D4 1.0>} maps to (<D3 'Trains'>, 'Trains')
Obs {<B5 '4xxxx'>} maps to (<B3 'Cars'>, 'Cars')
Obs {<C5 3.0>} maps to (<C3 'Planes'>, 'Planes')
Obs {<D5 2.0>} maps to (<D3 'Trains'>, 'Trains')
Obs {<B6 4.0>} maps to (<B3 'Cars'>, 'Cars')
Obs {<C6 1.0>} maps to (<C3 'Planes'>, 'Planes')
Obs {<D6 3.0>} maps to (<D3 'Trains'>, 'Trains')
Obs {<B7 2.0>} maps to (<B3 'Cars'>, 'Cars')
Obs {<C7 5.0>} maps to (<C3 'Planes'>, 'Planes')
Obs {<D7 5.0>} maps to (<D3 'Trains'>, 'Trains')


In [12]:
# Note that the return value of the h1.cellvalobs() function is actually a pair 
# composed of the heading cell reference and its value.  This is to help us 
# provide alternative values in the output in difficult cases.  

# The heading cells do not always contain the values you want them 
# to give in the final output.  

# For example, we can map its string value to something else, like so
h1.cellvalueoverride["Cars"] = "Horses"
for ob in obs:
    print("Obs", ob, "maps to", h1.cellvalobs(ob))

Obs {<B4 2.0>} maps to (<B3 'Cars'>, 'Horses')
Obs {<C4 2.0>} maps to (<C3 'Planes'>, 'Planes')
Obs {<D4 1.0>} maps to (<D3 'Trains'>, 'Trains')
Obs {<B5 '4xxxx'>} maps to (<B3 'Cars'>, 'Horses')
Obs {<C5 3.0>} maps to (<C3 'Planes'>, 'Planes')
Obs {<D5 2.0>} maps to (<D3 'Trains'>, 'Trains')
Obs {<B6 4.0>} maps to (<B3 'Cars'>, 'Horses')
Obs {<C6 1.0>} maps to (<C3 'Planes'>, 'Planes')
Obs {<D6 3.0>} maps to (<D3 'Trains'>, 'Trains')
Obs {<B7 2.0>} maps to (<B3 'Cars'>, 'Horses')
Obs {<C7 5.0>} maps to (<C3 'Planes'>, 'Planes')
Obs {<D7 5.0>} maps to (<D3 'Trains'>, 'Trains')


In [13]:
# Alternatively, you can override by the reference to a single cell to a value 
# rather than applying the mapping to its string value, like so:
c3cell = tab.excel_ref('C3')._cell
h1.cellvalueoverride[c3cell] = "Submarines"
for ob in obs:
    print("Obs", ob, "maps to", h1.cellvalobs(ob))

Obs {<B4 2.0>} maps to (<B3 'Cars'>, 'Horses')
Obs {<C4 2.0>} maps to (<C3 'Planes'>, 'Submarines')
Obs {<D4 1.0>} maps to (<D3 'Trains'>, 'Trains')
Obs {<B5 '4xxxx'>} maps to (<B3 'Cars'>, 'Horses')
Obs {<C5 3.0>} maps to (<C3 'Planes'>, 'Submarines')
Obs {<D5 2.0>} maps to (<D3 'Trains'>, 'Trains')
Obs {<B6 4.0>} maps to (<B3 'Cars'>, 'Horses')
Obs {<C6 1.0>} maps to (<C3 'Planes'>, 'Submarines')
Obs {<D6 3.0>} maps to (<D3 'Trains'>, 'Trains')
Obs {<B7 2.0>} maps to (<B3 'Cars'>, 'Horses')
Obs {<C7 5.0>} maps to (<C3 'Planes'>, 'Submarines')
Obs {<D7 5.0>} maps to (<D3 'Trains'>, 'Trains')


In [14]:
# Although in the above case we could have implemented by 
#    h1.cellvalueoverride["Planes"] = "Submarines", 
# this technique would have still worked if C3 was blank.  

# Sometimes in very complex cases it convenient to build 
# headings that contain blank cells, and then assign their 
# their values using cellvalueoverride map


In [15]:
# Finally, to handle the most extreme post processing cases, 
# you can override the header value for an 
# individual observation element to the value you want.  

# Here we map just the top cell in the B-column to a different value, 
# in place of its normal matching heading.  
# Since we don't get to a heading cell for obs B4, its slot is filled with None
b4cell = tab.excel_ref('B4')._cell
h1.cellvalueoverride[b4cell] = "Clouds"
for ob in obs:
    print("Obs", ob, "maps to", h1.cellvalobs(ob))

Obs {<B4 2.0>} maps to (None, 'Clouds')
Obs {<C4 2.0>} maps to (<C3 'Planes'>, 'Submarines')
Obs {<D4 1.0>} maps to (<D3 'Trains'>, 'Trains')
Obs {<B5 '4xxxx'>} maps to (<B3 'Cars'>, 'Horses')
Obs {<C5 3.0>} maps to (<C3 'Planes'>, 'Submarines')
Obs {<D5 2.0>} maps to (<D3 'Trains'>, 'Trains')
Obs {<B6 4.0>} maps to (<B3 'Cars'>, 'Horses')
Obs {<C6 1.0>} maps to (<C3 'Planes'>, 'Submarines')
Obs {<D6 3.0>} maps to (<D3 'Trains'>, 'Trains')
Obs {<B7 2.0>} maps to (<B3 'Cars'>, 'Horses')
Obs {<C7 5.0>} maps to (<C3 'Planes'>, 'Submarines')
Obs {<D7 5.0>} maps to (<D3 'Trains'>, 'Trains')


In [16]:
# We can use this feature to manage default no-lookup values, 
# for example, take this cell (outside what happens to be the observation
# area within the table) and see how its lookup into r1 according 
# to the rules in h1 gives an invalid (None) value
wob = tab.excel_ref('B1')
print("Wrong-Obs", wob, "maps to", h1.cellvalobs(ob))

Wrong-Obs {<B1 2014.0>} maps to (<D3 'Trains'>, 'Trains')


In [17]:
# But we can set the override to map None to a default value
h1.cellvalueoverride[None] = "Who knows?"
print("Wrong-Obs", wob, "maps to", h1.cellvalobs(ob))

Wrong-Obs {<B1 2014.0>} maps to (<D3 'Trains'>, 'Trains')


In [18]:
# This leads us onto a new feature, of a constant value dimension over the 
# whole obs set by referring to a None header set
h3 = HDim(None, "Category", cellvalueoverride={None:"Beatles"})
print("Wrong-Obs", wob, "maps to", h3.cellvalobs(ob))

Wrong-Obs {<B1 2014.0>} maps to (None, 'Beatles')


In [19]:
# Notice that you can set the cellvalueoverride dict in the constructor if you don't 
# mind having very long statements.  

# But in this special case of a constant dimension we have a helper function which 
# does exactly the same thing
h3 = HDim(None, "Category", cellvalueoverride={None:"Beatles"})
h3 = HDimConst("Category", "Beatles")
for ob in obs:
    print("Obs", ob, "maps to", h3.cellvalobs(ob))

Obs {<B4 2.0>} maps to (None, 'Beatles')
Obs {<C4 2.0>} maps to (None, 'Beatles')
Obs {<D4 1.0>} maps to (None, 'Beatles')
Obs {<B5 '4xxxx'>} maps to (None, 'Beatles')
Obs {<C5 3.0>} maps to (None, 'Beatles')
Obs {<D5 2.0>} maps to (None, 'Beatles')
Obs {<B6 4.0>} maps to (None, 'Beatles')
Obs {<C6 1.0>} maps to (None, 'Beatles')
Obs {<D6 3.0>} maps to (None, 'Beatles')
Obs {<B7 2.0>} maps to (None, 'Beatles')
Obs {<C7 5.0>} maps to (None, 'Beatles')
Obs {<D7 5.0>} maps to (None, 'Beatles')


In [20]:
# Let's put these all together into one list of dimensions and roll them into 
# nice big ConversionSegment, which we can preview
dimensions = [ 
    HDim(tab.excel_ref('B1'), TIME, CLOSEST, ABOVE), 
    HDim(r1, "Vehicles", DIRECTLY, ABOVE), 
    HDim(r2, "Name", DIRECTLY, LEFT), 
    HDimConst("Category", "Beatles")
]
c1 = ConversionSegment(tab, dimensions, obs)
savepreviewhtml(c1)


OBS,TIME,Vehicles,Name
Date,2014.0,,
,,,
,Cars,Planes,Trains
John,2.0,2.0,1.0
Paul,4xxxx,3.0,2.0
Ringo,4.0,1.0,3.0
George,2.0,5.0,5.0


In [21]:
# Try clicking on the OBS cells above to see how the cells connect to the headings

# Sometimes it's useful to see the raw values as they are looked up
for ob in c1.segment:
    print(c1.lookupobs(ob))

{'__y': 3, '__tablename': 'beatles', 'TIME': '2014.0', 'Category': 'Beatles', '__x': 1, 'Vehicles': 'Cars', 'OBS': 2.0, 'Name': 'John'}
{'__y': 3, '__tablename': 'beatles', 'TIME': '2014.0', 'Category': 'Beatles', '__x': 2, 'Vehicles': 'Planes', 'OBS': 2.0, 'Name': 'John'}
{'__y': 3, '__tablename': 'beatles', 'TIME': '2014.0', 'Category': 'Beatles', '__x': 3, 'Vehicles': 'Trains', 'OBS': 1.0, 'Name': 'John'}
{'OBS': 4.0, '__y': 4, '__tablename': 'beatles', 'TIME': '2014.0', 'Category': 'Beatles', '__x': 1, 'Vehicles': 'Cars', 'Name': 'Paul', 'DATAMARKER': 'xxxx'}
{'__y': 4, '__tablename': 'beatles', 'TIME': '2014.0', 'Category': 'Beatles', '__x': 2, 'Vehicles': 'Planes', 'OBS': 3.0, 'Name': 'Paul'}
{'__y': 4, '__tablename': 'beatles', 'TIME': '2014.0', 'Category': 'Beatles', '__x': 3, 'Vehicles': 'Trains', 'OBS': 2.0, 'Name': 'Paul'}
{'__y': 5, '__tablename': 'beatles', 'TIME': '2014.0', 'Category': 'Beatles', '__x': 1, 'Vehicles': 'Cars', 'OBS': 4.0, 'Name': 'Ringo'}
{'__y': 5, '__tab

In [22]:
# If you want to use pandas on this ConversionSegment, we have a handy function that does it
# (Pandas is very hard to learn, but very powerful indeed.  If you are going to be doing things 
# with data as part of your work for more than a year, it will pay back oodles if you 
# invest the time to master it.)
df = c1.topandas()
df


TIMEUNIT='Year'


,OBS,DATAMARKER,TIME,TIMEUNIT,Vehicles,Name,Category,__x,__y,__tablename
0,2.0,NaN,2014,Year,Cars,John,Beatles,1,3,beatles
1,2.0,NaN,2014,Year,Planes,John,Beatles,2,3,beatles
2,1.0,NaN,2014,Year,Trains,John,Beatles,3,3,beatles
3,4.0,xxxx,2014,Year,Cars,Paul,Beatles,1,4,beatles
4,3.0,NaN,2014,Year,Planes,Paul,Beatles,2,4,beatles
5,2.0,NaN,2014,Year,Trains,Paul,Beatles,3,4,beatles
6,4.0,NaN,2014,Year,Cars,Ringo,Beatles,1,5,beatles
7,1.0,NaN,2014,Year,Planes,Ringo,Beatles,2,5,beatles
8,3.0,NaN,2014,Year,Trains,Ringo,Beatles,3,5,beatles
9,2.0,NaN,2014,Year,Cars,George,Beatles,1,6,beatles


In [25]:
# Once we are done with the lookups, we can generate the technicalCSV for the WDA system
pL = c1.topandas()
L = writetechnicalCSV(None, c1).split("\n")
#print(L)
Lp = writetechnicalCSV(None, pL).split("\n")
L == Lp
writetechnicalCSV("example.csv", c1)
k = readtechnicalCSV("example.csv")
k[0]
pL


writing 1 conversion segments into /home/goatchurch/sensiblecode/src/databaker/databaker/tutorial/example.csv
conversionwrite segment size 12 table 'beatles; 


,OBS,DATAMARKER,TIME,TIMEUNIT,Vehicles,Name,Category,__x,__y,__tablename
0,2.0,NaN,2014,Year,Cars,John,Beatles,1,3,beatles
1,2.0,NaN,2014,Year,Planes,John,Beatles,2,3,beatles
2,1.0,NaN,2014,Year,Trains,John,Beatles,3,3,beatles
3,4.0,xxxx,2014,Year,Cars,Paul,Beatles,1,4,beatles
4,3.0,NaN,2014,Year,Planes,Paul,Beatles,2,4,beatles
5,2.0,NaN,2014,Year,Trains,Paul,Beatles,3,4,beatles
6,4.0,NaN,2014,Year,Cars,Ringo,Beatles,1,5,beatles
7,1.0,NaN,2014,Year,Planes,Ringo,Beatles,2,5,beatles
8,3.0,NaN,2014,Year,Trains,Ringo,Beatles,3,5,beatles
9,2.0,NaN,2014,Year,Cars,George,Beatles,1,6,beatles


In [24]:
# We almost always write this output to a file because the preview is so big and ugly
# You can replace the c1 with a list of ConversionSegments
writetechnicalCSV("example.csv", c1)




writing 1 conversion segments into /home/goatchurch/sensiblecode/src/databaker/databaker/tutorial/example.csv
conversionwrite segment size 12 table 'beatles; 


In [25]:
# You can save preview tables to a file as well, but it can only be of a single segment
# The #injblockXXXX thing is shown at the top of the file to help you make sure you are seeing
# the right preview, and not some earlier version.  
# (We'd like to make the link work, but it's difficult.  Also it would be nice if the page 
# you had automatically reloaded itself if it was already open, in order to cut down on the RSI)
savepreviewhtml(c1, "preview.html")

tablepart 'beatles' written #injblock1007
javascript calculated


In [26]:
# In summary, we have the capability of codelookups and nolookups with the cellvalueoverride dict
# We can look at individual observations and filter them.  
# You can also use a list instead of a bag when you make a conversion set, 
# and then sort that list

obslist = list(obs.unordered_cells)  # list(segment) otherwise gives bags of one element
obslist.sort(key=lambda cell: str(cell.value))
c2 = ConversionSegment(tab, dimensions, obslist)
topandas(c2)


,Category,DATAMARKER,Name,OBS,TIME,TIMEUNIT,Vehicles,tabname,x,y
0,Beatles,NaN,John,1.0,2014-01-01,Year,Trains,beatles,3,3
1,Beatles,NaN,Ringo,1.0,2014-01-01,Year,Planes,beatles,2,5
2,Beatles,NaN,John,2.0,2014-01-01,Year,Cars,beatles,1,3
3,Beatles,NaN,Paul,2.0,2014-01-01,Year,Trains,beatles,3,4
4,Beatles,NaN,George,2.0,2014-01-01,Year,Cars,beatles,1,6
5,Beatles,NaN,John,2.0,2014-01-01,Year,Planes,beatles,2,3
6,Beatles,NaN,Paul,3.0,2014-01-01,Year,Planes,beatles,2,4
7,Beatles,NaN,Ringo,3.0,2014-01-01,Year,Trains,beatles,3,5
8,Beatles,NaN,Ringo,4.0,2014-01-01,Year,Cars,beatles,1,5
9,Beatles,xxxx,Paul,4.0,2014-01-01,Year,Cars,beatles,1,4


In [27]:
# It's also obvious to see how to make load in multiple spreadsheets
# as well as output multiple files, by making multiple calls to
#    loadxlstabs(inputfilename)
#    writetechnicalCSV(outputfilename, [selected list of segments])
# with all the HDim, HDimConst and ConversionSegment creation stuff 
# in between.  

# But it's going to take some working out of conventions and habits 
# in order to avoid getting into a mess.  

# The main part of the design is to make it easy to preview what's going 
# on at every step of the way, which means you'll want to load everything 
# in at the start, and save everything out at the end.

# Unfortunately this is very memory intensive and might not work on the 
# largest sets of files.  However, a working system that is close to 
# that as possible is desirable, to avoid producing "efficient pipelines" 
# of data processing that are going to be a right pain to debug when you 
# come back to it.  

If you like, proceed to [Tutorial 2.5](tutorial2.5.ipynb).